In [ ]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install pandas openpyxl numpy scipy matplotlib

import pandas as pd
import numpy as np
import os
from zipfile import ZipFile as zf
import urllib as ur
import shutil
import tempfile
import datetime
import sqlalchemy as sa

#grab FAA tables
engine_types = {
'0' : 'None'
,'1' : 'Reciprocating'
,'2' : 'Turbo-prop'
,'3' : 'Turbo-shaft'
,'4' : 'Turbo-jet'
,'5' : 'Turbo-fan'
,'6' : 'Ramjet'
,'7' : '2 Cycle'
,'8' : '4 Cycle'
,'9' : 'Unknown'
,'10' : 'Electric'
,'11' : 'Rotary'
}

engine_types = {
'1' : 'Glider'
'2' : 'Balloon'
'3' : 'Blimp/Dirigible'
'4' : 'Fixed wing single engine'
'5' : 'Fixed wing multi engine'
'6' : 'Rotorcraft'
'7' : 'Weight-shift-control'
'8' : 'Powered Parachute'
'9' : 'Gyroplane'
'H' : 'Hybrid Lift'
'O' : 'Other'
}

In [6]:
file_dict = {
                2020: {'url':'https://registry.faa.gov/database/yearly/ReleasableAircraft.2020.zip'}
            }

file_dict.items()
filepath = r"C:\Users\Lyle.Tripp\Downloads\ReleasableAircraft.2020.zip"

## Initialize some connection string/dbs for SQL interaction
sql = {}

sql["AEDT_SERVER_NAME"] = r"localhost"#\MSSQLSERVER_2012" #\\MSSQLSERVER_2017"
sql["AEDT_SERVER_USER"] = 'lt_remote'
sql["AEDT_SERVER_PWD"] = 'Welcome12345^'
sql["CANDIDATE_FLEET_DATABASE"] = sql["LEGACY_FLEET_DATABASE"] = 'FAA_AIRCRAFT_REG' #legacy_fleet_database
#sql["DRIVER"] = '{SQL Server Native Client 11.0}' #
sql["DRIVER"] = '{ODBC Driver 17 for SQL Server}'
sql["USE_WINDOWS_AUTHENTICATION"] = False

if sql["USE_WINDOWS_AUTHENTICATION"]:
    cnxn_string = 'DRIVER='+sql["DRIVER"]+';PORT=1433;SERVER='+sql["AEDT_SERVER_NAME"]+';PORT=1443;DATABASE='+sql["LEGACY_FLEET_DATABASE"]+';trusted_connection=yes'
else:
    cnxn_string = 'DRIVER='+sql["DRIVER"]+';PORT=1433;SERVER='+sql["AEDT_SERVER_NAME"]+';PORT=1443;DATABASE='+sql["LEGACY_FLEET_DATABASE"]+';UID='+ sql["AEDT_SERVER_USER"]+';PWD='+ sql["AEDT_SERVER_PWD"]
params = urllib.parse.quote_plus(cnxn_string)
engine = sa.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

# for k,v in file_dict.items():
#     #download file to 
#     with ur.request.urlopen(v['url']) as response:
#         with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
#             shutil.copyfileobj(response, tmp_file)
with zf(filepath) as myzip:
    with myzip.open('ACFTREF.txt') as myfile:
        print(myzip.infolist())
        with myzip.open('ACFTREF.txt') as myfile:
            df_acft = pd.read_csv(myfile, index_col='CODE')
        with myzip.open('ENGINE.txt') as myfile:
            df_eng = pd.read_csv(myfile, index_col='CODE', dtype={'CODE':'str'})
        with myzip.open('MASTER.txt') as myfile:
            df_registry = pd.read_csv(
                myfile
                , dtype={'ENG MFR MDL':'Int64'}
                , na_values=['     ','        ']
                , parse_dates=['EXPIRATION DATE', 'CERT ISSUE DATE']
                , infer_datetime_format=True)
        with myzip.open('DOCINDEX.txt') as myfile:
            df_index = pd.read_csv(myfile)
        with myzip.open('ardata.pdf') as myfile:
            pass   

yes_engine = df_registry["ENG MFR MDL"].isna() == False
df1 = df_registry[yes_engine].join(df_acft, on='MFR MDL CODE').join(
    df_eng
    , on='ENG MFR MDL'
    , rsuffix='_engine'
    , lsuffix='_aircraft')

no_engine = df_registry["ENG MFR MDL"].isna() == True
df2 = df_registry[no_engine].join(df_acft, on='MFR MDL CODE')

df = pd.concat([df1, df2]) 

df.drop(columns=['MFR','MODEL'],axis=1, inplace=True)

df.to_sql('faa_aircraft_reg',con=engine)

[<ZipInfo filename='ardata.pdf' compress_type=deflate external_attr=0x20 file_size=507615 compress_size=434266>, <ZipInfo filename='ACFTREF.txt' compress_type=deflate file_size=13869362 compress_size=1597632>, <ZipInfo filename='ENGINE.txt' compress_type=deflate file_size=224659 compress_size=40339>, <ZipInfo filename='DEALER.txt' compress_type=deflate file_size=15672499 compress_size=729153>, <ZipInfo filename='RESERVED.txt' compress_type=deflate file_size=29879126 compress_size=2517683>, <ZipInfo filename='MASTER.txt' compress_type=deflate file_size=176822576 compress_size=28596470>, <ZipInfo filename='DOCINDEX.txt' compress_type=deflate file_size=5130492 compress_size=465376>, <ZipInfo filename='DEREG.txt' compress_type=deflate file_size=257468321 compress_size=33779235>]


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
df.dtypes

N-NUMBER                    object
SERIAL NUMBER               object
MFR MDL CODE                object
ENG MFR MDL                  Int64
YEAR MFR                    object
TYPE REGISTRANT             object
NAME                        object
STREET                      object
STREET2                     object
CITY                        object
STATE                       object
ZIP CODE                    object
REGION                      object
COUNTY                      object
COUNTRY                     object
LAST ACTION DATE             int64
CERT ISSUE DATE     datetime64[ns]
CERTIFICATION               object
TYPE AIRCRAFT               object
TYPE ENGINE                  int64
STATUS CODE                 object
MODE S CODE                  int64
FRACT OWNER                 object
AIR WORTH DATE             float64
OTHER NAMES(1)              object
OTHER NAMES(2)              object
OTHER NAMES(3)              object
OTHER NAMES(4)              object
OTHER NAMES(5)      

In [245]:
pd.to_datetime(df["EXPIRATION DATE"], format='%Y%m%d')

ValueError: time data '        ' does not match format '%Y%m%d' (match)

In [244]:
df["EXPIRATION DATE"].

0    20230430
1    20220228
2    20201031
5    20210228
6    20220131
Name: EXPIRATION DATE, dtype: object